How to set up selenium and client?

In [ ]:
var importer = require('../Core');

var client, results, getCredentials, loginGoogle, getAllXPath;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client',
        'decrypt password',
        'log in Google',
        'get all elements'
    ])
    .then(r => {
        results = r;
        return r[0].runInNewContext();
    })
    .then(r => {
        client = r();
        getCredentials = results[1].runInNewContext({client});
        loginGoogle = results[2].runInNewContext({client, getCredentials});
        getAllXPath = results[3].runInNewContext({client});
        return client;
    });
};
module.exports = getClient;
getClient;



In [ ]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


How to download google timeline data using webdriver?

In [ ]:
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
    .url('https://www.google.com/maps/timeline')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
               .then(url => url.indexOf('timeline') > -1, 20000, ''))
    .catch(function (e) {
        console.log(e);
    })
    .pause(3000)
    .click('button[jsaction="header.select-today"]')
    .pause(2000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

var readAllPages = () => {
    // TODO: add stop conditions
    var oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(
        Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    dayKey = '';
    var promises = [];
    for (var c = 0; c < days; c++) {
        var result;
        promises[promises.length] = (resolve) => readTimelinePage()
            .then(r => result = r)
            .click('.previous-date-range-button')
            .pause(2000)
            .then(() => {
                if(typeof(result) == 'undefined') {
                    resolve();
                }
                console.log(result);
                fs.writeFileSync(
                    PROJECT_PATH + '/timeline-' + result[0].timeline + '_.json',
                    JSON.stringify(result, null, 4))
                resolve(result);
            })
    }
    return promises;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


In [ ]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var readTimelinePage = () => {
    return client.getAllXPath({
        day: '//*[contains(@class,"timeline-subtitle")]//text()',
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
    .then(r => {
        console.log(r.day);
        if(typeof r.day != 'string') {
            return;
        }
        var currDate = chrono.parseDate(r.day);
        var newKey = currDate.getDate() 
            + months[currDate.getMonth()] 
            + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        
        return r.items.map(i => {
            var timelineData = (i.data + '').split(':');
            var start, end, length;
            if(timelineData[0]) {
                start = new Date(parseFloat(timelineData[1]));
                end = new Date(parseFloat(timelineData[2]));
                length = end.getTime() - start.getTime();
            } else {
                start = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[0]);
                end = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[1]);
                length = 0;
            }
            return ({
                type: 'timeline',
                timeline: dayKey,
                name: i.title,
                location: i.location,
                time: start,
                length: isNaN(length) ? 0 : length
            })
        })
    })
    .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


In [ ]:
var fs = require('fs');
var importer = require('../Core');

$$.async()
client.getGoogleTimeline()
    .then((r) => importer.runAllPromises(readAllPages()))
    .catch(e => $$.sendError(e));



Load location export data?



In [2]:
var importer = require('../Core');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var dayKey;

var loadLocations = (file) => {
    var locationCache = {};
    return importer.streamJson(file, [true, {emitPath: true}], (match) => {
        if(match.path[0] === 'locations') {
            var currDate = new Date(parseInt(match.value.timestampMs));
            var newKey = currDate.getDate() + months[currDate.getMonth()]
                + (currDate.getFullYear() + '').substr(2, 2);
            if (newKey != dayKey) {
                dayKey = newKey;
            }
            var newRow = Object.assign({
                    time: currDate,
                    type: 'location',
                    location: dayKey,
            }, match.value);
            var cache = locationCache[newRow.location];
            if(typeof cache === 'undefined') {
                cache = locationCache[newRow.location] = [];
            }
            cache[cache.length] = newRow;
        }
    })
    .then(() => locationCache)
};
module.exports = loadLocations;
loadLocations;



[Function: loadLocations]

Find the average latitute and longitude at each destination?



In [4]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Timeline';

function toRadians (angle) {
  return angle * (Math.PI / 180);
}

var straightDistance = (lat1, lon1, lat2, lon2) => {
    var R = 6371e3; // metres
    var φ1 = toRadians(lat1);
    var φ2 = toRadians(lat2);
    var Δφ = toRadians(lat2-lat1);
    var Δλ = toRadians(lon2-lon1);

    var a = Math.sin(Δφ/2) * Math.sin(Δφ/2) +
            Math.cos(φ1) * Math.cos(φ2) *
            Math.sin(Δλ/2) * Math.sin(Δλ/2);
    var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));

    var d = R * c;
    return d;
}

var averageDestinations = locationCache => {
    return importer.runAllPromises(Object.keys(locationCache).map(date => resolve => {
        console.log(date);
        if(!fs.existsSync(PROJECT_PATH + '/timeline-' + date + '_.json')) {
            return resolve();
        }
        var timelineData = JSON.parse(
            fs.readFileSync(PROJECT_PATH + '/timeline-' + date + '_.json')
            .toString());
        var destinations = [];
        for(const d of timelineData) {
            if((d.name + '') == '' || (d.name + '').indexOf('Driving') > -1) {
                continue;
            }
            const nearest = _.sortBy(
                locationCache[date],
                l => Math.abs(new Date(l.time).getTime() - new Date(d.time).getTime()))
                .slice(0, 3);
            // make sure it isn't off by much
            const averageLat = nearest.map(n => n.latitudeE7)
                .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
            const averageLon = nearest.map(n => n.longitudeE7)
                .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
            if(nearest.filter(nearby => straightDistance(
                    nearby.latitudeE7 / 10000000,
                    nearby.longitudeE7 / 10000000,
                    averageLat,
                    averageLon
                ) > 500).length > 0) {
                destinations.push(Object.assign(d, {
                    averageLat: averageLat,
                    averageLon: averageLon,
                    locations: nearest.map(nearby => Object.assign(nearby, {
                        averageLat: nearby.latitudeE7 - averageLat,
                        averageLon: nearby.longitudeE7 - averageLon,
                        averageDist: straightDistance(
                            nearby.latitudeE7 / 10000000,
                            nearby.longitudeE7 / 10000000,
                            averageLat,
                            averageLon
                        )
                    }))
                }));
            }
            destinations.push(Object.assign(d, {
                averageLat: averageLat,
                averageLon: averageLon
            }));
        }
        resolve([destinations]);
    }))
}
module.exports = averageDestinations;
averageDestinations;



[Function: averageDestinations]

Reconcile timeline data with calendar?



In [ ]:
var importer = require('../Core');

var getEvents = (time) => {
    const start = new Date(time);
    start.setHours(0, 0, 0);
    const end = new Date(time);
    end.setHours(23, 59, 0);
    return listEvents({
        auth: options.auth,
        timeMin: start.toString(),
        timeMax: end.toString()
    })
    .catch(e => console.log(e));
}

var filterEvents = (events, locations) => {
    const unmatched = [], matches = [];
    for(const l of locations) {
        const matching = events.filter(e => {
            return ((l.name + '').indexOf('Gainey') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('Swiftpage') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Drive home')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Work from home')
            || (typeof e.event.location !== 'undefined'
                && ((e.event.location + '').indexOf(l.location + '') > -1
                || (e.event.location + '').indexOf(l.name + '') > -1))
        });
        if(matching.length == 0) {
            unmatched.push(l);
        } else {
            matches.push(Object.assign(l, matching[0]));
        }
    }
    console.log(unmatched.length + ' - '
                + unmatched.map(u => u.name + '').join(', '));
}

var listEvents, options = {};
var reconcileTimeline = (destinations) => {
    var placesNearby, getOauthClient, events, locations = [];
    return importer.interpretAll([
        'import google calendar api',
        'places nearby google maps',
        'list events',
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
        placesNearby = r[1].runInNewContext();
        listEvents = r[2].runInNewContext();
        return getOauthClient(options);
    })
    .then(() => getEvents(destinations[0].time))
    .then(r => {
        events = r;
        return importer.runAllPromises(destinations.map(d => resolve => {
            console.log(d);
            return placesNearby(d.name + ', ' + d.location,
                            {lat: d.averageLat, lng: d.averageLon})
            .then(r => {
                console.log(r[0])
                resolve(Object.assign(d, r[0]))
            })
            .catch(e => resolve(e))
        }))
    })
    .then(r => {
        locations = r;
        return filterEvents(events, locations);
    })
    .catch(e => console.log(e))
}
module.exports = reconcileTimeline;
reconcileTimeline;



In [ ]:
var importer = require('../Core');

var loadLocations, averageDestinations, reconcileTimeline;

$$.async();
importer.interpretAll([
    'load locations export data',
    'average latitude longitude',
    'reconcile timeline calendar'
])
.then(r => {
    loadLocations = r[0].runInNewContext();
    averageDestinations = r[1].runInNewContext();
    reconcileTimeline = r[2].runInNewContext();
})
.then(() => {
    return loadLocations('/Users/briancullinan/Downloads/Takeout 7/Location History/Location History.json');
})
.then(locationCache => averageDestinations({
    '20Sep17': locationCache['20Sep17'],
    '19Sep17': locationCache['19Sep17'],
    '18Sep17': locationCache['18Sep17']
}))
.then(destinations => {
    return importer.runAllPromises(destinations.map(d => resolve => {
        return reconcileTimeline(d);
    }))
})
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



20Sep17
19Sep17
18Sep17
{ type: 'timeline',
  timeline: '20Sep17',
  name: '6934 E Sandra Terrace',
  location: 'Scottsdale, AZ 85254',
  time: '2017-09-20T14:53:00.000Z',
  length: 0,
  averageLat: 33.6345342,
  averageLon: -111.93046436666667 }
timeout
{ type: 'timeline',
  timeline: '20Sep17',
  name: 'Gainey Ranch Corporate Center',
  location: 'Swiftpage',
  time: '2017-09-20T15:09:00.000Z',
  length: 0,
  averageLat: 33.565729733333335,
  averageLon: -111.91911803333332,
  locations: 
   [ { time: 2017-09-20T15:09:02.000Z,
       type: 'location',
       location: '20Sep17',
       timestampMs: '1505920142000',
       latitudeE7: 335651861,
       longitudeE7: -1119128894,
       accuracy: 95,
       velocity: 2,
       heading: 322,
       altitude: 383,
       activity: [Object],
       averageLat: 335651827.43427026,
       averageLon: -1119128782.080882,
       averageDist: 580.2631635315644 },
     { time: 2017-09-20T15:07:58.000Z,
       type: 'location',
       location: '